<h2>Define the network</h2>

In [1]:
# defines an nn

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        # n_c^(l - 1), n_c^(l), f
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        
        # y = Wx + b
        # in-dimension, out-d
        self.fc1 = nn.Linear(16 * 6 * 6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # backward func: automatically defined using autograd
        
#         x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
#         # If the size is a square you can only specify a single number
#         x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        
#         x = x.view(-1, self.num_flat_features(x))
        
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
#         x = self.fc3(x)

        z1 = self.conv1(x)
        a1 = F.relu(z1)
        a1 = F.max_pool2d(a1, (2, 2))
        
        z2 = self.conv2(a1)
        a2 = F.relu(z2)
        a2 = F.max_pool2d(a2, 2)
        a2 = a2.view(-1, self.num_flat_features(a2))
        
        z3 = self.fc1(a2)
        a3 = F.relu(z3)
        
        z4 = self.fc2(a3)
        a4 = F.relu(z4)
        
        a5 = self.fc3(a4)
        
        return a5
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [3]:
params = list(net.parameters())

print(len(params))
for param in params:
    print(param.size())

10
torch.Size([6, 1, 3, 3])
torch.Size([6])
torch.Size([16, 6, 3, 3])
torch.Size([16])
torch.Size([120, 576])
torch.Size([120])
torch.Size([84, 120])
torch.Size([84])
torch.Size([10, 84])
torch.Size([10])


10  
  
torch.Size([6, 1, 3, 3])  # W.size = (n_c^(l + 1), n_c^(l), f_h, f_w)  
torch.Size([6])  # b  
torch.Size([16, 6, 3, 3])  
torch.Size([16])  
  
torch.Size([120, 576])  # W  
torch.Size([120])  # b  
torch.Size([84, 120])  
torch.Size([84])  
torch.Size([10, 84])  
torch.Size([10])

In [4]:
# processes inputs

In [5]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.0624,  0.0130,  0.0016,  0.1018, -0.0311,  0.0188, -0.0906,  0.0009,
         -0.0091, -0.1172]], grad_fn=<AddmmBackward>)


In [6]:
# calls backward

In [7]:
# Zero the gradient buffers of all parameters
net.zero_grad()
# inits with random gradients
out.backward(torch.randn(1, 10))

<h2>Loss Function</h2>

In [8]:
# computes the loss

In [9]:
output = net(input)
target = torch.rand(10).view(1, -1)

criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.4273, grad_fn=<MseLossBackward>)


In [10]:
# when we call loss.backward(), the whole graph is differentiated 
# w.r.t. the loss, and all Tensors in the graph that has 
# requires_grad=True will have their .grad Tensor **accumulated** 
# with the gradient.
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0])
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])

outputs of the next_functions can be:  
&nbsp;&nbsp;&nbsp;(<AccumulateGrad object at 0x7fb6451fa2d0>, 0): learning param leaves  
&nbsp;&nbsp;&nbsp;(None, 0): const leaves  
&nbsp;&nbsp;&nbsp;(<MulBackward0 object at 0x7fb6451fa210>, 0): operations  

<h2>Backprop</h2>

In [11]:
# You need to clear the existing gradients though, 
# else gradients will be accumulated to existing gradients.

In [12]:
net.zero_grad()

print(net.conv1.weight.grad)
print(list(net.parameters())[0].grad)

print(net.conv1.bias.grad)
print(list(net.parameters())[1].grad)

print()

loss.backward()

print(net.conv1.weight.grad)
print(list(net.parameters())[0].grad)

print(net.conv1.bias.grad)
print(list(net.parameters())[1].grad)

tensor([[[[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]],


        [[[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]],


        [[[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]],


        [[[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]],


        [[[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]],


        [[[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]]])
tensor([[[[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]],


        [[[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]],


        [[[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]],


        [[[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]],


        [[[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]],


        [[[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]]])
tensor([0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0.])

tensor([[[[-0.0070, -0.

<h2>Update the weights</h2>

In [13]:
# updates the weights

In [14]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.01)

# in the training loop!

# forward prop
output = net(input)

loss = criterion(output, target)

# back prop
optimizer.zero_grad()  # zeroes the gradient buffers!
# Observe how gradient buffers had to be manually set to zero 
# using optimizer.zero_grad(). This is because gradients are 
# accumulated
loss.backward()

# updates params using gd
# Performs a single optimization step.
optimizer.step()